In [18]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

# Display all of the columns
pd.set_option('display.max_columns', None)

In [19]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path('data/board_games_complete.csv')
)
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [20]:
# Process data
df['category_clean'] = df['category_clean'].replace({'Animals': 'Other','Card Game': 'Other', 'Dice': 'Other','Electronic': 'Other','Game System': 'Other','Horror': 'Other','Humor': 'Other','Mature / Adult': 'Other','Miniatures': 'Other','Music': 'Other','none': 'Other','Party Game': 'Other','Religious': 'Other','Fighting': 'Other','Racing': 'Other','Sports': 'Other','Age of Reason': 'Era','American West': 'Era','Ancient': 'Era','Medieval': 'Era','Napoleonic': 'Era','Post-Napoleonic': 'Era','Prehistoric': 'Era','Renaissance': 'Era','Abstract Strategy': 'Strategy','Bluffing': 'Strategy','City Building': 'Strategy','Civilization': 'Strategy','Collectible Components': 'Strategy','Deduction': 'Strategy','Economic': 'Strategy','Exploration': 'Strategy','Farming': 'Strategy','Industry / Manufacturing': 'Strategy','Mafia': 'Strategy','Maze': 'Strategy','Murder/Mystery': 'Strategy','Negotiation': 'Strategy','Pirates': 'Strategy','Political': 'Strategy','Puzzle': 'Strategy','Real-time': 'Strategy','Spies/Secret Agents': 'Strategy','Territory Building': 'Strategy','Trains': 'Strategy','Transportation': 'Strategy','Zombies': 'Strategy','Action / Dexterity': 'Adventure','Adventure': 'Adventure','Arabian': 'Adventure','Aviation / Flight': 'Adventure','Comic Book / Strip': 'Adventure','Fantasy': 'Adventure','Movies / TV / Radio theme': 'Adventure','Mythology': 'Adventure','Nautical': 'Adventure','Pike and Shot': 'Adventure','Print & Play': 'Adventure','Science Fiction': 'Adventure','Space Exploration': 'Adventure','Travel': 'Adventure','Video Game Theme': 'Adventure','Book': 'Educational',"Children's Game": 'Educational','Educational': 'Educational','Environmental': 'Educational','Math': 'Educational','Medical': 'Educational','Memory': 'Educational','Novel-based': 'Educational','Number': 'Educational','Trivia': 'Educational','Word Game': 'Educational','American Civil War': 'Wargame','American Indian Wars': 'Wargame','American Revolutionary War': 'Wargame','Civil War': 'Wargame','Korean War': 'Wargame','Modern Warfare': 'Wargame','Vietnam War': 'Wargame','Wargame':'Wargame','World War I':'Wargame','World War II':'Wargame',})

In [21]:
# Process data
df['mechanic_clean'] = df['mechanic_clean'].replace({'Cooperative Play': 'Co-op',
'Dice Rolling': 'Dice_Rolling',
'Roll / Spin and Move': 'Dice_Rolling',
'Simultaneous Action Selection': 'PVP',
'Chit-Pull System': 'PVP',
'Campaign / Battle Card Driven': 'PVP',
'Hex-and-Counter': 'PVP',
'Area Control / Area Influence': 'PVP',
'Action / Movement Programming': 'PVP',
'Variable Player Powers': 'PVP',
'Betting/Wagering': 'PVP',
'Trick-taking': 'PVP',
'Player Elimination': 'PVP',
'Take That': 'PVP',
'Secret Unit Deployment': 'PVP',
'Rock-Paper-Scissors': 'PVP',
'Area Movement': 'Movement',
'Grid Movement': 'Movement',
'Modular Board': 'Movement',
'Tile Placement': 'Movement',
'Line Drawing': 'Movement',
'Pattern Recognition': 'Movement',
'Area Enclosure': 'Movement',
'Route/Network Building': 'Movement',
'Pick-up and Deliver': 'Movement',
'Crayon Rail System': 'Movement',
'Point to Point Movement': 'Movement',
'Time Track': 'Movement',
'none': 'Other',
'Action Point Allowance System': 'Other',
'Hand Management': 'Other',
'Worker Placement': 'Other',
'Pattern Building': 'Other',
'Auction/Bidding': 'Other',
'Card Drafting': 'Other',
'Paper-and-Pencil': 'Other',
'Press Your Luck': 'Other',
'Deck / Pool Building': 'Other',
'Set Collection': 'Other',
'Partnerships': 'Other',
'Commodity Speculation': 'Other',
'Area-Impulse': 'Other',
'Voting': 'Other',
'Stock Holding': 'Other',
'Memory': 'Other',
'Variable Phase Order': 'Other',
'Trading': 'Other',
'Storytelling': 'Role_Playing',
'Simulation': 'Role_Playing',
'Role Playing': 'Role_Playing',
'Acting': 'Role_Playing',
'Singing': 'Role_Playing',
})

In [22]:
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,PVP,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Other,2,Educational,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Other,3,Other,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Other,6,Strategy,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,PVP,3,Wargame,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  5000 non-null   int64  
 1   bgg_url               5000 non-null   object 
 2   id                    5000 non-null   int64  
 3   name                  5000 non-null   object 
 4   minplayers            5000 non-null   int64  
 5   maxplayers            5000 non-null   int64  
 6   playingtime           5000 non-null   int64  
 7   minplaytime           5000 non-null   int64  
 8   maxplaytime           5000 non-null   int64  
 9   yearpublished         5000 non-null   int64  
 10  average_rating        5000 non-null   float64
 11  geek_rating           5000 non-null   float64
 12  users_rated           5000 non-null   int64  
 13  image_url             5000 non-null   object 
 14  thumb_url             5000 non-null   object 
 15  age                  

In [24]:
# drop unwanted columns
df = df.drop(['bgg_url',
'yearpublished',
'geek_rating',
'users_rated',
'image_url',
'thumb_url',
'mechanic',
'category',
'designer',
'publisher',
'expands',
'reimplements',
'type',
'minage',
'bayes_average_rating',
'total_traders',
'total_wanters',
'total_wishers',
'total_comments',
'average_weight',
'name',
'playingtime',
'minplaytime',
'num_fans',
'designer_count',
], axis=1)


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            5000 non-null   int64  
 1   id              5000 non-null   int64  
 2   minplayers      5000 non-null   int64  
 3   maxplayers      5000 non-null   int64  
 4   maxplaytime     5000 non-null   int64  
 5   average_rating  5000 non-null   float64
 6   age             5000 non-null   int64  
 7   total_owners    5000 non-null   int64  
 8   total_weights   5000 non-null   float64
 9   mechanic_count  5000 non-null   int64  
 10  mechanic_clean  5000 non-null   object 
 11  category_count  5000 non-null   int64  
 12  category_clean  5000 non-null   object 
 13  designer_clean  5000 non-null   object 
 14  name_clean      4988 non-null   object 
dtypes: float64(2), int64(9), object(4)
memory usage: 586.1+ KB


In [26]:
df = df.dropna()

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4988 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            4988 non-null   int64  
 1   id              4988 non-null   int64  
 2   minplayers      4988 non-null   int64  
 3   maxplayers      4988 non-null   int64  
 4   maxplaytime     4988 non-null   int64  
 5   average_rating  4988 non-null   float64
 6   age             4988 non-null   int64  
 7   total_owners    4988 non-null   int64  
 8   total_weights   4988 non-null   float64
 9   mechanic_count  4988 non-null   int64  
 10  mechanic_clean  4988 non-null   object 
 11  category_count  4988 non-null   int64  
 12  category_clean  4988 non-null   object 
 13  designer_clean  4988 non-null   object 
 14  name_clean      4988 non-null   object 
dtypes: float64(2), int64(9), object(4)
memory usage: 623.5+ KB


In [28]:
ohe_df = pd.get_dummies(df, 
                        columns = ['category_clean', 'mechanic_clean', 'designer_clean' ], 
                        prefix = ['category_clean', 'mechanic_clean', 'designer_clean' ])
print(ohe_df)

      rank      id  minplayers  maxplayers  maxplaytime  average_rating  age  \
0        1  174430           1           4          120         8.91603   12   
1        2  161936           2           4           60         8.65105   13   
2        3  182028           2           4          120         8.55251   14   
3        4  167791           1           5          120         8.39916   12   
4        5   12333           2           2          180         8.32843   13   
...    ...     ...         ...         ...          ...             ...  ...   
4995  4996    4288           2           2          120         7.26169   12   
4996  4997    1965           2           6          100         6.43691   10   
4997  4998  223518           1           4           60         6.53060   13   
4998  4999  144479           2           6           20         7.59369   10   
4999  5000  154519           2           4           30         7.95414   10   

      total_owners  total_weights  mech

In [29]:
ohe_df.to_csv("boardgames.csv", index=False)

In [ ]:
 # select features for the model
features = ['category_clean', 'mechanic_clean', 'total_weights', 'age', 'average_rating', 'maxplaytime', 'minplayers', 'maxplayers',]

# create a dataframe with only the relevant features
df_model = ohe_df[['beer_id', 'beer_name', 'beer_style', 'ibu_category'] + features + ['abv_category', 'brewery_name', 'city', 'state']]
df = df[['beer_name', 'beer_style', 'abv_category', 'ibu_category', 'availability', 'brewery_name', 'city', 'state', 'types']]

In [ ]:
user_preferences = np.array([user_preference_category, user_preference_mechanic, user_preference_complexity, user_preference_age, user_preference_minplayers, user_preference_maxplayers, user_preference_playingtime]) 
# User's preferences or ratings

In [ ]:
# Number of neighbors to consider (you can change this) 
k_neighbors = 5 

In [ ]:
# Initialize the k-NN model 
knn = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean') 
# You can choose a different distance metric 

In [ ]:
# Fit the model to your data 
knn.fit(data[:, 1:]) 
# Exclude the item IDs when fitting the model 

In [ ]:
# Find the k nearest neighbors for user's preferences distances, 
indices = knn.kneighbors([user_preferences])

In [ ]:
# Recommend items based on the nearest neighbors 
recommended_items = data[indices[0], 0] 
# Get the item IDs of the recommended items 


In [ ]:
# You can print or return the recommended items 
print("Recommended Items:", recommended_items) 